# Notebook trainer cheatsheet: API and CLI

- Automation page
- Recovering the API KEY (Automation page, User page, RestClient)

In [ ]:
from pymisp import ExpandedPyMISP
from pprint import pprint
AUTHKEY = "ptU1OggdiLLWlwHPO9B3lzpwEND3hL7gH0uEsyYL"
URL = "http://127.0.0.1:8080"

def print_result(result):
    flag_printed = False
    if isinstance(result, list):
        print("Count: %s" % len(result))
        flag_printed = True
        for i in res:
            if 'Event' in i and 'Attribute' in i['Event']:
                print("  - Attribute count: %s" % len(i['Event']['Attribute']))
    elif isinstance(result, dict):
        if 'Attribute' in result:
            print("Count: %s" % len(result['Attribute']))
            flag_printed = True
        elif 'Event' in result and 'Attribute' in result['Event']['Attribute']:
            print("Attribute count: %s" % len(result['Event']['Attribute']))
            flag_printed = True
    if flag_printed:
        print('----------')
    pprint(result)

# Events

## Creation and Edition

In [ ]:
# Creation
misp_url = URL + '/events/add'
relative_path = ''

body = {
    "info": "Event created via the API",
    "threat_level_id": 1,
    "distribution": 0
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Edition 1
misp_url = URL + '/events/edit/'
relative_path = '29'

body = {
    "distribution": 4,
    "sharing_group_id": 1
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)  
print_result(res)

In [ ]:
# Edition 2 - Adding Attribute
misp_url = URL + '/events/edit/'
relative_path = '29'

body = {
    "distribution": 0,
    "Attribute": [
        {
            "value": "9.9.9.9",
            "type": "ip-src"
        }
    ]
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Edition 2 - tagging - The bad way (Fetch the whole event and re-process everything)
misp_url = URL + '/events/edit/'
relative_path = '29'

body = {
    "distribution": 0,
    "EventTag": {
        "Tag": [
            {"name":"tlp:red"}
        ]
    }
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Edition 2 - tagging - The better way
misp_url = URL + '/tags/attachTagToObject'
relative_path = ''

body = {
    "uuid": "5ce3c1c7-7820-45db-9dba-67a6bf06e1fe", # can be anything: event or attribute
    "tag": "Android Malware"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searching the Event index (Move it to the search topic)
misp_url = URL + '/events/index'
relative_path = ''

body = {
    "eventinfo": "api",
    "publish_timestamp": "10d",
    "org": "ORGNAME"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searching the Event index
misp_url = URL + '/events/index'
relative_path = ''

body = {
    "hasproposal": 1,
    "tag": ["tlp:amber"]
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)

print('Event number: %s' % len(res))
print_result(res)

# Attributes

## Creation and edition

In [ ]:
event_id = 29

In [ ]:
# Adding
misp_url = URL + '/attributes/add/'
relative_path = str(event_id)

body = {
    "value": "8.8.8.9",
    "type": "ip-dst"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Adding invalid attribute type
misp_url = URL + '/attributes/add/'
relative_path = str(event_id)

body = {
    "value": "8.8.8.9",
    "type": "md5"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Editing
misp_url = URL + '/attributes/edit/'
relative_path = '36586'

body = {
    "value": "127.0.0.1",
    "to_ids": 0,
    "comment": "Comment added via the API",
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Editing with data taken from JSON views. 
# <!> (timestamp) contrast the difference with PyMISP
misp_url = URL + '/attributes/edit/'
relative_path = '36584'

body = {
    "id": "36584",
    "type": "ip-src",
    "category": "Network activity",
    "to_ids": 1,
    "uuid": "5ce404bb-23f0-421c-842f-79a9bf06e1fe",
    "event_id": "29",
    "distribution": "5",
    "comment": "",
    "sharing_group_id": "0",
    "deleted": 0,
    "disable_correlation": 0,
    "object_id": "0",
    "object_relation": None,
    "value": "1.1.1.1",
    "Galaxy": [],
    "ShadowAttribute": []
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

# Objects

In [ ]:
# Example of un-documented endpoint
misp_url = URL + '/objects/add/'
relative_path = str(event_id)

body = {
    "name": "microblog",
    "meta-category": "misc",
    "description": "Microblog post like a Twitter tweet or a post on a Facebook wall.",
    "template_uuid": "8ec8c911-ddbe-4f5b-895b-fbff70c42a60",
    "template_version": "5",
    "event_id": event_id,
    "timestamp": "1558702173",
    "distribution": "5",
    "sharing_group_id": "0",
    "comment": "",
    "deleted": False,
    "ObjectReference": [],
    "Attribute": [
        {
            "type": "text",
            "category": "Other",
            "to_ids": False,
            "event_id": event_id,
            "distribution": "5",
            "timestamp": "1558702173",
            "comment": "",
            "sharing_group_id": "0",
            "deleted": False,
            "disable_correlation": False,
            "object_relation": "post",
            "value": "post",
            "Galaxy": [],
            "ShadowAttribute": []
        }
    ]
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

## RestSearch
**Aka: Most powerfull search tool in MISP**

### RestSearch - Attributes

In [ ]:
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searches on Attribute's data
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "type": "ip-src",
    "value": "8.8.%"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searches on Attribute's data
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "deleted": [0, 1]    # Consider both deleted AND not deleted
}

# [] == {"OR": []}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searches on Attribute's data
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "tags": "tlp:white",
#     "tags": ["tlp:white", "tlp:green"]
#     "tags": ["tlp:white", "!tlp:green"]
#     "tags": "tlp:%"
#     "includeEventTags": 1
        # BRAND NEW (only tag)! Prefered way (Most accurate): Distinction between OR and AND!
#     "tags": {"AND": ["tlp:green", "Malware"], "NOT": ["%ransomware%"]}
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Paginating
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "page": 1,
    "limit": 2
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searches based on time: Absolute
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "from": "2019/05/21" # or "2019-05-21"
    # from and to NOT REALLY USEFULL.. 
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searches based on time: Relative
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

# /!\ Last: works on the publish_timestamp -> may be confusing
# Units: days, hours, minutes and secondes
body = {
    "returnFormat": "json",
    "eventid": event_id,
    "to_ids": 1,
    "last": "10d"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

## Precision regarding the different timestamps
- ``publish_timestamp`` = Time at which the event was published
    - Usage: get data that arrived in my system since x
    - E.g.: New data from a feed
- ``timestamp`` = Time of the last modification on the data
    - data was modified in the last x hours
    - E.g.: Last updated data from a feed
- ``event_timestamp``: Used in the Attribute scope
    - Event modified in the last x hours

In [ ]:
# Searches with attachments
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
    "type": "attachment",
#     "withAttachments": 1
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searches - Others
misp_url = URL + '/attributes/restSearch/'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": event_id,
#     "enforceWarninglist": 1
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

### RestSearch - Events

In [ ]:
# Searching using the RestSearch
misp_url = URL + '/events/restSearch'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventid": 29,
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searching using the RestSearch - Other return format
!curl \
 -d '{"returnFormat":"yara","eventid":29}' \
 -H "Authorization: ptU1OggdiLLWlwHPO9B3lzpwEND3hL7gH0uEsyYL" \
 -H "Accept: application/json" \
 -H "Content-type: application/json" \
 -X POST 127.0.0.1:8080/events/restSearch 2> /dev/null

In [ ]:
# Searching using the RestSearch - Other return format
!curl \
 -d '{"returnFormat":"csv","eventid":29}' \
 -H "Authorization: ptU1OggdiLLWlwHPO9B3lzpwEND3hL7gH0uEsyYL" \
 -H "Accept: application/json" \
 -H "Content-type: application/json" \
 -X POST 127.0.0.1:8080/events/restSearch 2> /dev/null

In [ ]:
# Searching using the RestSearch - Filtering
misp_url = URL + '/events/restSearch'
relative_path = ''

body = {
    "returnFormat": "json",
    "value": "fast-decay-model.json"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searching using the RestSearch
misp_url = URL + '/events/restSearch'
relative_path = ''

body = {
    "returnFormat": "json",
    "org": "CIRCL",
    "metadata": 1
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searching using the RestSearch
misp_url = URL + '/events/restSearch'
relative_path = ''

body = {
    "returnFormat": "json",
    "eventinfo": "%via the API%",
    "published": 1
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

# Sightings

In [ ]:
# Creating sightings
misp_url = URL + '/sightings/add'
relative_path = ''

body = {
    "id": "36589"
#     "value": "fast-decay-model.json"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Searching for sighted elements
misp_url = URL + '/sightings/restSearch/event'
relative_path = ''

body = {
    "returnFormat": "json",
    "id": 29,
#     "includeAttribute": 1,
#     "includeEvent": 1
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

# Warning lists

In [ ]:
# Checking values against the warining list
misp_url = URL + '/warninglists/checkValue'
relative_path = ''

body = ["8.8.8.8", "yolo"]

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

# Instance management

In [ ]:
# Creating Organisation
misp_url = URL + '/admin/organisations/add'
relative_path = ''

body = {
    "name": "TEMP_ORG"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Creating Users
misp_url = URL + '/admin/users/add'
relative_path = ''

body = {
    "email": "from_api@admin.test",
    "org_id": 2,
    "role_id": 3,
    "termsaccepted": 1,
    "change_pw": 0, # User prompted to change the psswd once logged in
    "password": "~~UlTrA_SeCuRe_PaSsWoRd~~"
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Creating Sharing Groups
misp_url = URL + '/sharing_groups/add'
relative_path = ''

body = {
    "name": "TEMP_SG",
    "releasability": "To nobody",
    "SharingGroupOrg": [
        {
            "name": "ORGNAME",
            "extend": 1
        },
        {
            "name": "CIRCL",
            "extend": 1
        }
    ]
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Server
misp_url = URL + '/servers/add'
relative_path = ''

body = {
    "url": "http://127.0.0.1:80/",
    "name": "Myself",
    "remote_org_id": "2",
    "authkey": "UHwmZCH4QdSKqPVunxTzfSes8n7ibBhUlsd0dmx9"
    
}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Server settings
misp_url = URL + '/servers/serverSettings'
relative_path = ''

body = {}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)

In [ ]:
# Statistics
misp_url = URL + '/users/statistics'
relative_path = ''

body = {}

misp = ExpandedPyMISP(misp_url, AUTHKEY, False)
res = misp.direct_call(relative_path, body)
print_result(res)